# Optimizing Using Pandas DataFrame

### Solution for Stack Overflow question
### 2018-09-13

https://stackoverflow.com/questions/52305104/optimizing-using-pandas-data-frame/52310584#52310584

In [ ]:
import pandas as pd
import numpy as np
import datetime
from haversine import haversine

In [ ]:
def main_function(df, row):
    """
    The main difference here is that everything is vectorized
    Returns: DataFrame
    """
    
    df_new = pd.DataFrame()
    df_storage = pd.DataFrame()
    
    pos_datetime = df.POSDATETIME.isin([row['POSDATETIME']])  # creates a Boolean map
    array_len = len(pos_datetime)
    new_index = pos_datetime.index

    df_new['StormID'] = df.loc[pos_datetime, 'STORMID']
    df_new['ModelBaseTime'] = df.loc[pos_datetime, 'MODELDATETIME']
    df_new['Model'] = df.loc[pos_datetime, 'MODEL']
    df_new['Tau'] = df.loc[pos_datetime, 'TAU']
    
    # Distance
    df_new['LatCARQ'] = pd.DataFrame(np.full((array_len, 1), row['LAT']), index=new_index).loc[pos_datetime, 0]
    df_new['LonCARQ'] = pd.DataFrame(np.full((array_len, 1), row['LON']), index=new_index).loc[pos_datetime, 0]
    df_new['LatModel'] = df.loc[pos_datetime, 'LAT']
    df_new['LonModel'] = df.loc[pos_datetime, 'LON']

    
    def calc_dist_error(row):
        return round(haversine((row['LatCARQ'], row['LonCARQ']), (row['LatModel'], row['LonModel']), miles=True)) if row['LatModel'] != 0.0 else None
    
    df_new['DistError'] = df_new.apply(calc_dist_error, axis=1)
    
    # Wind
    df_new['WindCARQ'] = pd.DataFrame(np.full((array_len, 1), row['WIND']), index=new_index).loc[pos_datetime, 0]
    df_new['WindModel'] = df.loc[pos_datetime, 'WIND']
    df_storage['row_WIND'] = pd.DataFrame(np.full((array_len, 1), row['WIND']), index=new_index).loc[pos_datetime, 0]
    df_storage['df_WIND'] = df.loc[pos_datetime, 'WIND']
    
    
    def wind_error_calc(row):
        return (row['row_WIND'] - row['df_WIND']) if row['df_WIND'] != 0 else None

    df_new['WindError'] = df_storage.apply(wind_error_calc, axis=1)
    
    # Air Pressure
    df_new['PresCARQ'] = pd.DataFrame(np.full((array_len, 1), row['PRES']), index=new_index).loc[pos_datetime, 0]
    df_new['PresModel'] = df.loc[pos_datetime, 'PRES']
    df_storage['row_PRES'] = pd.DataFrame(np.full((array_len, 1), row['PRES']), index=new_index).loc[pos_datetime, 0]
    df_storage['df_PRES'] = df.loc[pos_datetime, 'PRES']

    
    def pres_error_calc(row):
        return abs(row['row_PRES'] - row['df_PRES']) if row['df_PRES'] != 0 else None

    df_new['PresError'] = df_storage.apply(pres_error_calc, axis=1)
    
    del(df_storage)

    return df_new


In [ ]:
def calculate_adeck_errors(in_file):
    """
    Retruns: DataFrame
    """
    print(f'Starting Data Calculations: {datetime.datetime.now().strftime("%I:%M:%S%p on %B %d, %Y")}')
    pd.set_option('max_columns', 20)
    pd.set_option('max_rows', 300)
          
    # read in the raw csv
    adeck_df = pd.read_csv(in_file)
    adeck_df['MODELDATETIME'] = pd.to_datetime(adeck_df['MODELDATETIME'], format='%Y-%m-%d %H:%M')
    adeck_df['POSDATETIME'] = pd.to_datetime(adeck_df['POSDATETIME'], format='%Y-%m-%d %H:%M')
    
    #extract only the carq items and remove duplicates
    carq_data = adeck_df[(adeck_df.MODEL == 'CARQ') & (adeck_df.TAU == 0)].drop_duplicates(keep='last')    
    print('Len carq_data: ', len(carq_data))
          
    #remove carq items from original
    final_df = adeck_df[adeck_df.MODEL != 'CARQ']
    print('Len final_df: ', len(final_df))
                  
    df_out_new = pd.DataFrame()

    for index, row in carq_data.iterrows():

        test_df = main_function(final_df, row)  # function call

        df_out_new = df_out_new.append(test_df, sort=False)
        

    df_out_new = df_out_new.reset_index(drop=True)
    df_out_new = df_out_new.where((pd.notnull(df_out_new)), None)
    
    print(f'Finishing Data Calculations: {datetime.datetime.now().strftime("%I:%M:%S%p on %B %d, %Y")}')
    return df_out_new


In [ ]:
in_file = 'data/aal062018.csv'

In [ ]:
df = calculate_adeck_errors(in_file)

In [ ]:
len(df)

In [ ]:
df.head(20)

# Original Code

In [ ]:
def calculate_adeck_errors(in_file):
    print(f'Starting Data Calculations: {datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y")}')
    pd.set_option('display.max_columns', 12)

    # read in the raw csv
    adeck_df = pd.read_csv(in_file)
    #print(adeck_df)

    #extract only the carq items and remove duplicates
    carq_data = adeck_df[(adeck_df.MODEL == 'CARQ') & (adeck_df.TAU == 0)].drop_duplicates(keep='last')
    #print(carq_data)

    #remove carq items from original
    final_df = adeck_df[adeck_df.MODEL != 'CARQ']
    #print(final_df)

    row_list = []
    for index, row in carq_data.iterrows():
        position_time = row['POSDATETIME']
        for index, arow in final_df.iterrows():
            if arow['POSDATETIME'] == position_time:
                # match, so do calculations
                storm_id = arow['STORMID']
                model_base_time = arow['MODELDATETIME']
                the_hour = arow['TAU']
                the_model = arow['MODEL']
                point1 = float(row['LAT']), float(row['LON'])
                point2 = float(arow['LAT']), float(arow['LON'])
                if arow['LAT'] == 0.0:
                    dist_error = None
                else:
                    dist_error = int(round(haversine(point1, point2, miles=True)))

                if arow['WIND'] != 0:
                    wind_error = int(abs(int(row['WIND']) - int(arow['WIND'])))
                else: wind_error = None

                if arow['PRES'] != 0:
                    pressure_error = int(abs(int(row['PRES']) - int(arow['PRES'])))
                else:
                    pressure_error = None

                lat_carq = row['LAT']
                lon_carq = row['LON']
                lat_model = arow['LAT']
                lon_model = arow['LON']
                wind_carq = row['WIND']
                wind_model = arow['WIND']
                pres_carq = row['PRES']
                pres_model = arow['PRES']

                row_list.append([storm_id, model_base_time, the_model, the_hour, lat_carq, lon_carq, lat_model, lon_model, dist_error,
                             wind_carq, wind_model, wind_error, pres_carq, pres_model, pressure_error])

    result_df = pd.DataFrame(row_list)
    result_df = result_df.where((pd.notnull(result_df)), None)
    result_cols = ['StormID', 'ModelBasetime', 'Model' , 'Tau',
               'LatCARQ', 'LonCARQ', 'LatModel', 'LonModel', 'DistError',
               'WindCARQ', 'WindModel','WindError',
               'PresCARQ', 'PresModel','PresError']

    result_df.columns = result_cols
          
    return result_df

in_file = 'data/aal062018.csv'
df = calculate_adeck_errors(infile)